# Data Visualization

The primary tools we'll use for plotting come from the [Holoviz](https://holoviz.org/) family of Python libraries, principally [GeoViews](https://geoviews.org/) and [hvPlot](https://hvplot.holoviz.org/). These are largely built on top of [HoloViews](https://holoviews.org/) and support multiple backends for rendering plots (notably [Bokeh](http://bokeh.pydata.org/) for interactive visualization and [Matplotlib](http://matplotlib.org/) for static, publication-quality plots).

## [GeoViews](https://geoviews.org/)

From the [GeoViews documentation](https://geoviews.org/index.html):

> GeoViews is a [Python](http://python.org/) library that makes it easy to explore and visualize geographical, meteorological, and oceanographic datasets, such as those used in weather, climate, and remote sensing research.
>
> GeoViews is built on the [HoloViews](http://holoviews.org/) library for building flexible visualizations of multidimensional data. GeoViews adds a family of geographic plot types based on the [Cartopy](http://scitools.org.uk/cartopy) library, plotted using either the [Matplotlib](http://matplotlib.org/) or [Bokeh](http://bokeh.pydata.org/) packages. With GeoViews, you can now work easily and naturally with large, multidimensional geographic datasets, instantly visualizing any subset or combination of them, while always being able to access the raw data underlying any plot.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

import geoviews as gv
gv.extension('bokeh')
from geoviews import opts

### Displaying a basemap

- Principal utility is `gv.tile_sources`
- Use the method `opts` to specify optional settings
- Bokeh menu at right enables interactive exploration

In [ ]:
basemap = gv.tile_sources.OSM.opts(width=600, height=400)
basemap

### Plotting points

In [ ]:
tokyo_lonlat = (139.692222, 35.689722)
print(tokyo_lonlat)

+ `geoviews.Points` accepts a list of tuples (each of the form `(x, y)`) to plot.
+ Use the OpenStreetMap tiles from `gv.tile_sources.OSM` as a basemap.
+ Overlay using the Holoviews operator `*`
+ Define the options using `geoviews.opts`
+ ? find a way to initialize the zoom level sensibly?

In [ ]:
tokyo_point   = gv.Points([tokyo_lonlat])
point_opts = opts.Points(
                          size=48,
                          alpha=0.5,
                          color='red'
                        )

In [ ]:
# Use Holoviews * operator to overlay plot on basemap
# Note: zoom out to see basemap (starts zoomed "all the way in")
(basemap * tokyo_point).opts(point_opts)

In [ ]:
# to avoid starting zoomed all the way in, this zooms "all the way out"
(basemap * tokyo_point).opts(point_opts, opts.Overlay(global_extent=True))

### Plotting rectangles

+ Standard way to represent rectangle with corners
  $$(x_{\mathrm{min}},y_{\mathrm{min}}), (x_{\mathrm{min}},y_{\mathrm{max}}), (x_{\mathrm{max}},y_{\mathrm{min}}), (x_{\mathrm{max}},y_{\mathrm{max}})$$
  (assuming $x_{\mathrm{max}}>x_{\mathrm{min}}$ & $y_{\mathrm{max}}>y_{\mathrm{min}}$) is as a single 4-tuple
  $$(x_{\mathrm{min}},y_{\mathrm{min}},x_{\mathrm{max}},y_{\mathrm{max}}),$$
  i.e., the lower,left corner coordinates followed by the upper, right corner coordinates.

In [ ]:
# simple utility to make a rectangle of "half-width" dx & "half-height" dy & centred pt
def bounds(pt,dx,dy):
    '''Returns rectangle represented as tuple (x_lo, y_lo, x_hi, y_hi)
    given inputs pt=(x, y), half-width & half-height dx & dy respectively,
    where x_lo = x-dx, x_hi=x+dx, y_lo = y-dy, y_hi = y+dy.
    '''
    return tuple(coord+sgn*delta for sgn in (-1,+1) for coord,delta in zip(pt, (dx,dy)))

In [ ]:
# Verify that the function bounds works as intended
marrakesh_lonlat = (-7.93, 31.67)
dlon, dlat = 0.25, 0.25
marrakesh_rect = bounds(marrakesh_lonlat, dlon, dlat)
print(marrakesh_rect)

+ `geoviews.Rectangles` accepts a list of bounding boxes (each described by a tuple of the form `(x_min, y_min, x_max, y_max)`) to plot.

In [ ]:
rect_opts = opts.Rectangles(
                                line_width=0,
                                alpha=0.25,
                                color='red'
                            )

In [ ]:
rectangle = gv.Rectangles([marrakesh_rect])
(basemap * rectangle).opts( rect_opts )

We'll use the approach above to visualize *Areas of Interest (AOIs)* when constructing search queries for NASA EarthData products. In particular, the convention of representing a bounding box by (left, lower, right, upper) ordinates is also used in the [PySTAC](https://pystac.readthedocs.io/en/stable/) API.

---

## [hvPlot](https://hvplot.holoviz.org/)

+ [hvPlot](https://hvplot.holoviz.org/) is designed to extend the `.plot` API from Pandas DataFrames.
+ It works for Pandas DataFrames and Xarray DataArrays/Datasets.

### hvplot.pandas

The code below loads a Pandas DataFrame of tempereature data.

In [ ]:
import pandas as pd, numpy as np
from pathlib import Path
LOCAL_PATH = Path().cwd() / '..' / 'assets' / 'temperature.csv'

In [ ]:
df = pd.read_csv(LOCAL_PATH, index_col=0, parse_dates=[0])
df.head()

Let's extract a subset of columns from this DataFrame and generate a plot.

In [ ]:
west_coast = df[['Vancouver', 'Portland', 'San Francisco', 'Seattle', 'Los Angeles']]
west_coast.head()

The Pandas DataFrame `.plot` API provides access to a number of plotting methods. Here, we'll use `.plot.line`, but a range of other options are available (e.g., `.plot.area`, `.plot.bar`, `.plot.hist`, `.plot.scatter`, etc.). This API has been repeated in several libraries due to its convenience.

In [ ]:
west_coast.plot.line();

By importing `hvplot.pandas`, a similar interactive plot can be generated. The API for `.hvplot` mimics that for `.plot`. For instance, we can generate the line plot above using `.hvplot.line`. In this case, the default plotting backend is Bokeh, so the plot is *interactive*.

In [ ]:
import hvplot.pandas
west_coast.hvplot.line()

It is possible to modify the plot to make it cleaner.

In [ ]:
west_coast.hvplot.line(width=600, height=300, grid=True)

The `hvplot` API also works when chained together with other DataFrame method calls. For instance, we can resample the temperature data and compute a mean to smooth it out.

In [ ]:
smoothed = west_coast.resample('2d').mean()
smoothed.hvplot.line(width=600, height=300, grid=True)

### hvplot.xarray

The Pandas `.plot` API was also extended to Xarray, i.e., for Xarray `DataArray`.

In [ ]:
import xarray as xr
import hvplot.xarray
import rioxarray as rio

To start, load a local GeoTIFF file using `rioxarray` into an Zarray `DataArray` structure.

In [ ]:
LOCAL_PATH = Path('..') / 'assets' / 'OPERA_L3_DIST-ALERT-HLS_T10TEM_20220815T185931Z_20220817T153514Z_S2A_30_v0.1_VEG-ANOM-MAX.tif'

In [ ]:
data = rio.open_rasterio(LOCAL_PATH)
data

We do some minor reformatting to the `DataArray`.

In [ ]:
data = data.squeeze() # to reduce 3D array with singleton dimension to 2D array
data = data.rename({'x':'easting', 'y':'northing'})
data

The `.plot` API by default uses Matplotlib's `pcolormesh` to display a 2D array stored within a `DataArray`. This takes a little time to render for this moderately high-resolution image.

In [ ]:
data.plot(); # by default, uses pcolormesh

Again, the `.hvplot` API mimics the `.plot` API; by default, it uses a subclass derived from `holoviews.element.raster.Image`.

In [ ]:
plot = data.hvplot() # by default uses Image class
print(f'{type(plot)=}')
plot

+ The result above is an interactive plot, rendered using Bokeh.
+ It still needs cleaning up; notice the axes are transposed.

We can add some options to speed up rendering. We also make the call to `hvplot.image` explicit & specify the sequence of axes.

In [ ]:
image_opts = dict(rasterize=True, dynamic=True)

In [ ]:
plot = data.hvplot.image(x='easting', y='northing', **image_opts)
plot

Let's add options to fix the image size.

In [ ]:
image_opts.update(frame_width=500, frame_height=500, aspect='equal')

plot = data.hvplot.image(x='easting', y='northing', **image_opts)
plot

Next, let's change the image colormap and use transparency (`alpha`).

In [ ]:
image_opts.update( cmap='hot_r', clim=(0,100), alpha=0.8 )
plot = data.hvplot.image(x='easting', y='northing', **image_opts)
plot

Before adding a basemap, we need to account for the coordinate system. This is stored in the GeoTIFF file and, when read using `rioxarray.open_rasterio`, it is is preserved in the attribute `data.rio.crs`.

In [ ]:
crs = data.rio.crs
crs

We can use the CRS recovered above as an optional argument to `hvplot.image`. Notice the coordinates have changed on the axes, but the labels are wrong. We can fix that shortly.

In [ ]:
image_opts.update(crs=crs)

plot = data.hvplot.image(x='easting', y='northing', **image_opts)
plot

Let's now fix the labels. We'll use the Holoviews/GeoViews `opts` system to specify these options.

In [ ]:
label_opts = dict(title='VEG_ANOM_MAX', xlabel='Longitude (degrees)', ylabel='Latitude (degrees)')
plot = data.hvplot.image(x='easting', y='northing', **image_opts).opts(**label_opts)
plot

Let's overlay the image on a basemap so we can see the terrain underneath.

In [ ]:
base = gv.tile_sources.ESRI
base * plot

Finally, the white pixels are distracting; let's filter them out using the `DataArray` method `where`.

In [ ]:
plot = data.where(data>0).hvplot.image(x='easting', y='northing', **image_opts).opts(**label_opts)
plot * base